# TimeSHAP

#### Import

In [7]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import mat4py as mpy
import os
# model from jason
from tensorflow.keras.models import model_from_json
import matplotlib.pyplot as plt

from Load_and_Preprocess_Aachen import preprocess_aachen_dataset

from LSTM_Model_Training import load_model_structure_and_weights

from LSTM_Model_Training import plot_predictions_vs_actual, plot_residuals


## Loading and Preprocessing Data

#### Variables

In [8]:
# Define the file path for the dataset
file_path = '/Users/sigurdgjerdingen/Student/Master kode/Master_Herstad-Gjerdingen/Aachen/Degradation_Prediction_Dataset_ISEA.mat'

# Define the number of test cells to be used
test_cell_count = 3

# Set the random state for reproducibility
random_state = 42

In [9]:
# Load and preprocess Aachen data
preprocessed_full = preprocess_aachen_dataset(
    file_path,
    test_cell_count=3,
    random_state=42,
    phase=None,
    log_transform=False
)

# Assign feature splits to LSTM-specific variables
X_train_lstm = preprocessed_full["X_train"]
X_val_lstm   = preprocessed_full["X_val"]
X_test_lstm  = preprocessed_full["X_test"]

# Also extract the target variables
y_train = preprocessed_full["y_train"]
y_val   = preprocessed_full["y_val"]
y_test  = preprocessed_full["y_test"]

# Print the shapes to verify the assignments
print("  X_train_lstm:", X_train_lstm.shape)
print("  X_val_lstm:  ", X_val_lstm.shape)
print("  X_test_lstm: ", X_test_lstm.shape)
print("  y_train:", y_train.shape)
print("  y_val:  ", y_val.shape)
print("  y_test: ", y_test.shape)

  X_train_lstm: (7190, 272, 1)
  X_val_lstm:   (1798, 272, 1)
  X_test_lstm:  (595, 272, 1)
  y_train: (7190,)
  y_val:   (1798,)
  y_test:  (595,)


## Load model

In [10]:
# Load the model structure and weights from the specified file
model = load_model_structure_and_weights('model_20250131_123220')

# Display the model summary to verify the structure and loaded weights
model

Checking for model files in the following paths:
Structure file: Aachen/Models/model_20250131_123220.structure.json
Weights file: Aachen/Models/model_20250131_123220.weights.h5
Contents of directory 'Aachen/Models':
['model_20250127_135003.structure.json', 'model_20250131_124416.weights.h5', 'model_20250127_135003.weights.h5', 'model_20250128_130021.structure.json', 'model_20250128_130021.weights.h5', 'model_20250131_123220.structure.json', 'model_20250131_124416.structure.json', 'model_20250131_123220.weights.h5']
Model loaded from Aachen/Models/model_20250131_123220.structure.json and Aachen/Models/model_20250131_123220.weights.h5


<Sequential name=sequential_1, built=True>

## Test model

In [11]:
# Evaluate the model on the test set
test_loss, test_mae = model.evaluate(X_test_lstm, y_test_norm, verbose=1)
print(f"\nTest Loss: {test_loss}")
print(f"Test MAE: {test_mae}")

# Make predictions on the test set
y_pred = model.predict(X_test_lstm)

# Rescale predictions and test data back to the original range
y_pred_rescaled = y_pred.flatten() * y_max
y_test_rescaled = y_test_norm * y_max

# Compare actual and predicted values
results = pd.DataFrame({
    "Actual RUL80": y_test_rescaled,
    "Predicted RUL80": y_pred_rescaled
})
print(results.head())

# Plot predictions vs actual
plot_predictions_vs_actual(y_test_rescaled, y_pred_rescaled)

# Plot residuals
plot_residuals(y_test_rescaled, y_pred_rescaled)

# Print model summary
print(model.summary())

NameError: name 'y_test_norm' is not defined

## TimeSHAP

### Model entry point

In [ ]:
# Define the model prediction function
f = lambda x: model.predict(x)

### Baseline event

In [ ]:
def avg_rul(X_train_lstm):
    # Reshape the array to 2D for easier processing
    X_train_lstm_2d = X_train_lstm.reshape(-1, X_train_lstm.shape[-1])
    
    # Create a mask to exclude rows that are all zeros
    non_zero_mask = np.any(X_train_lstm_2d != 0, axis=1)
    
    # Calculate the mean of the non-zero rows
    avg = np.mean(X_train_lstm_2d[non_zero_mask])
    
    # Return the average as a NumPy array with shape (1, 1)
    return np.array([[avg]])

average_event = avg_rul(X_train_lstm)

average_event

array([[0.6730101]], dtype=float32)

In [ ]:
from timeshap.utils import get_avg_score_with_avg_event
avg_score_over_len = get_avg_score_with_avg_event(f, average_event, top=272)

avg_score_over_len

NameError: name 'average_event' is not defined

### Local explanations

#### Sequence to explain

In [50]:
# Select a random sequence from the test set

random_index = np.random.randint(0, len(X_test_lstm))
random_sequence = X_test_lstm[random_index][np.newaxis, ...]

# Number of 0 in random sequence
n_zeros = np.sum(random_sequence == 0)
n_zeros

225

#### Local report 

In [51]:
from timeshap.explainer import local_report

#pruning_dict = {'tol': 0.025}
pruning_dict = None
event_dict = {'rs': 42, 'nsamples': 32000}
feature_dict = {'rs': 42, 'nsamples': 32000, 'feature_names': None, 'plot_features': None}
cell_dict = {'rs': 42, 'nsamples': 32000, 'top_x_feats': 1, 'top_x_events': 5}
local_report(f, random_sequence, pruning_dict, event_dict, feature_dict, cell_dict=cell_dict, baseline=average_event)

Assuming all features are model features
No pruning dict passed. Skipping pruning procedures
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


/opt/miniconda3/envs/timeshap_env/lib/python3.9/site-packages/altair/utils/core.py:384: FutureWarning:

the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.

/opt/miniconda3/envs/timeshap_env/lib/python3.9/site-packages/altair/utils/core.py:384: FutureWarning:

the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.

/opt/miniconda3/envs/timeshap_env/lib/python3.9/site-packages/altair/utils/core.py:384: FutureWarning:

the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.

/opt/miniconda3/envs/timeshap_env/lib/python3.9/site-packages/altair/utils/core.py:384: FutureWarning:

the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser

alt.HConcatChart(...)

### Global explanations

#### Global report

In [12]:
# Sampel out 5 random sequences from the test set
random_indices = np.random.choice(len(X_test_lstm), 5)
random_sequences = X_test_lstm[random_indices]

In [13]:
from timeshap.explainer import global_report

model_feature = ["rul"]
schema = ["rul"]
plot_featrues = {"rul": "RUL"}

pruning_dict = {'tol': [0.05, 0.075], 'path': 'outputs/prun_all_tf.csv'}
event_dict = {'path': 'outputs/event_all_tf.csv', 'rs': 42, 'nsamples': 32000}
feature_dict = {'path': 'outputs/feature_all_tf.csv', 'rs': 42, 'nsamples': 32000, 'feature_names': None, 'plot_features': None,}

prun_stats, global_plot = global_report(
    f,                     # your model callable
    random_sequences,      # your NumPy array data
    pruning_dict,
    event_dict,
    feature_dict,
    average_event,              # baseline computed above
    model_features=None,        # provided as a list ["price"]
    schema=None,                # provided schema, also ["price"]
    entity_col=None,       # not applicable for NumPy array input
    time_col=None,         # not applicable here
    append_to_files=False,
    max_instances=1000,
    verbose=True
)

prun_stats

Assuming all features are model features
Calculating pruning algorithm
No time col provided, assuming dataset is ordered ascendingly by date
Allowed importance for pruned events: None
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
len 0 | importance -0.1946868598461151
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
len -1 | importance -0.17362748086452484
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
len -2 | importance -0.14873306453227997
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
len -3 | importance -0.12387983500957489
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
len -4 | importance -0.10195152461528778
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━

ValueError: Shape of passed values is (2730, 3), indices imply (2730, 4)

In [14]:
global_plot

NameError: name 'global_plot' is not defined

### Individual Plots

#### Local Plots

In [15]:
from timeshap.explainer import local_pruning
from timeshap.plot import plot_temp_coalition_pruning

pruning_dict = {'tol': 0,}
coal_plot_data, coal_prun_idx = local_pruning(f, random_sequence, pruning_dict, average_event)
# coal_prun_idx is in negative terms
pruning_idx = random_sequence.shape[1] + coal_prun_idx

pruning_plot = plot_temp_coalition_pruning(coal_plot_data, coal_prun_idx, plot_limit=280)
pruning_plot

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.


alt.LayerChart(...)

In [16]:
from timeshap.explainer import local_event
from timeshap.plot import plot_event_heatmap
pruning_idx = 0

event_dict = {'rs': 42, 'nsamples': 32000}
event_data = local_event(f, random_sequence, event_dict, entity_uuid=None, entity_col=None, baseline=average_event, pruned_idx=pruning_idx)
event_plot = plot_event_heatmap(event_data)
event_plot

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step


the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.


alt.LayerChart(...)

In [17]:
from timeshap.explainer import local_feat
from timeshap.plot import plot_feat_barplot

feature_dict = {'rs': 42, 'nsamples': 32000, 'feature_names': None, 'plot_features': None}
feature_data = local_feat(f, 
                          random_sequence, 
                          feature_dict,
                          entity_uuid=None, 
                          entity_col=None,
                          baseline=average_event, 
                          pruned_idx=pruning_idx)

feature_plot = plot_feat_barplot(feature_data, feature_dict.get('top_feats'), feature_dict.get('plot_features'))
feature_plot

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.


alt.LayerChart(...)

In [18]:
from timeshap.explainer import local_cell_level
from timeshap.plot import plot_cell_level

cell_dict = {'rs': 42, 'nsamples': 32000, 'top_x_events': 5, 'top_x_feats': 1}
cell_data = local_cell_level(f, random_sequence, cell_dict, event_data, feature_data, entity_uuid=None, entity_col=None, baseline=average_event, pruned_idx=pruning_idx)
feat_names = list(feature_data['Feature'].values)[:-1] # exclude pruned events
cell_plot = plot_cell_level(cell_data, feat_names, feature_dict.get('plot_features'))
cell_plot

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.


alt.LayerChart(...)

In [27]:
# Add sequence IDs (if 'Cell' exists, we can use that)
df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

# Create DataFrames from the NumPy arrays
def create_dataframe(X, y, original_df, dataset_type):
    data_list = []
    for i in range(len(X)):
        sequence = X[i].flatten()  # Flatten the sequence for DataFrame representation
        sequence_length = np.count_nonzero(sequence)  # Actual non-padded sequence length
        cell_id = original_df.iloc[i]["Cell"] if "Cell" in original_df.columns else f"{dataset_type}_{i}"
        
        data_list.append({
            'sequence_id': cell_id,
            'sequence_length': sequence_length,
            'sequence': sequence,
            'RUL_normalized': y[i]
        })
    
    return pd.DataFrame(data_list)

# Create DataFrames
df_train_final = create_dataframe(X_train_lstm, y_train_norm, df_train, 'train')
df_val_final = create_dataframe(X_val_lstm, y_val_norm, df_val, 'val')
df_test_final = create_dataframe(X_test_lstm, y_test_norm, df_test, 'test')


In [28]:
from typing import Callable, Union
import numpy as np
import pandas as pd
from timeshap.explainer.kernel import TimeShapKernel

def local_event_level_single_feature_standalone(
    f: Callable[[np.ndarray], np.ndarray],
    data: Union[pd.DataFrame, np.ndarray],
    event_dict: dict,
    baseline: Union[pd.DataFrame, np.ndarray],
    pruned_idx: int
) -> pd.DataFrame:
    """
    Computes event-level (cell-level) explanations for a single-feature model using
    the easier-to-use event mode of TimeShapKernel.
    
    Each time step (after the pruned index) is treated as an event.
    
    Parameters
    ----------
    f : Callable[[np.ndarray], np.ndarray]
        Model function that accepts a 3-D numpy array of shape (1, seq_len, 1)
        and returns a 2-D numpy array.
    data : Union[pd.DataFrame, np.ndarray]
        Input instance to be explained. If a DataFrame, it will be converted to a
        3-D numpy array.
    event_dict : dict
        Dictionary with parameters for the event explanation. Expected keys include:
          - 'rs': random seed (default: 42)
          - 'nsamples': number of samples for the kernel estimation (default: 100)
    baseline : Union[pd.DataFrame, np.ndarray]
        Baseline data (e.g. an average event) used for integrating out features.
    pruned_idx : int
        The index at which the sequence is pruned (e.g. 0 if no pruning is applied).
        
    Returns
    -------
    pd.DataFrame
        A DataFrame with columns ['Event', 'Shapley Value'] listing the explanation
        for each event (time step).
    """
    # Convert DataFrame to numpy array if needed.
    if isinstance(data, pd.DataFrame):
        data = data.to_numpy()
        # If 2-D, expand to 3-D (1, seq_len, num_features)
        if len(data.shape) == 2:
            data = np.expand_dims(data, axis=0)
        # Ensure that the last dimension is 1 (since we have one feature)
        if data.shape[-1] != 1:
            data = data[..., np.newaxis]
    
    # Extract parameters for the explainer from event_dict.
    random_seed = event_dict.get('rs', 42)
    nsamples = event_dict.get('nsamples', 100)
    
    # Instantiate the TimeShapKernel in event mode.
    # In event mode, the kernel automatically uses all time steps (after pruned_idx)
    # as the events to be explained.
    explainer = TimeShapKernel(f, baseline, random_seed, mode="event")
    
    # Compute Shapley values for the instance.
    # The output is an array of one value per event.
    shap_values_arr = explainer.shap_values(data, pruning_idx=pruned_idx, nsamples=nsamples)
    
    # In event mode, internally the explainer computes:
    #     self.varyingInds = np.arange(data.shape[1]-1, pruned_idx-1, -1)
    # so that the first computed value corresponds to the last time step.
    # To list events in natural order (from first event to last), we reverse the array.
    shap_values_arr_reversed = shap_values_arr[::-1]
    
    # Create labels for events.
    # Events correspond to the time steps starting at pruned_idx up to the end of the sequence.
    events = [f"Event {i}" for i in range(pruned_idx, data.shape[1])]
    
    # Build the explanation DataFrame.
    explanation_df = pd.DataFrame({
        "Event": events,
        "Shapley Value": shap_values_arr_reversed
    })
    
    return explanation_df


# Example usage:
if __name__ == "__main__":
   # Minimal event configuration.
    event_dict = {
        'rs': 42,          # Random seed.
        'nsamples': 10000    # Number of samples for the kernel estimation.
    }
    
    pruned_idx = 0  # No pruning in this simple example.
    
    # Get the event-level explanation.
    event_explanation = local_event_level_single_feature_standalone(
        f=f,
        data=random_sequence,
        event_dict=event_dict,
        baseline=average_event,
        pruned_idx=pruned_idx
    )
    
    print("Standalone Event-level Explanation (Single Feature):")
    print(event_explanation)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
Standalone Event-level Explanation (Single Feature):
         Event  Shapley Value
0      Event 0       0.000000
1      Event 1       0.000000
2      Event 2       0.000000
3      Event 3      -0.002441
4      Event 4       0.000000
..         ...            ...
267  Event 267      -0.003033
268  Event 268       0.000000
269  Event 269       0.000000
270  Event 270       0.000000
271  Event 271      -0.005808

[272 rows x 2 columns]


In [29]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Extract numeric event number from the Event column
event_explanation['Event Number'] = event_explanation['Event'].str.extract(r'Event (\d+)').astype(int)


sequence_line = random_sequence.flatten()

# --- Create Dual Y-axis Plot ---
# Create a subplot with secondary y-axis enabled.
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add the Shapley Value trace on the primary y-axis.
fig.add_trace(
    go.Scatter(
        x=event_explanation["Event Number"],
        y=event_explanation["Shapley Value"],
        mode='lines',
        name='Shapley Value'
    ),
    secondary_y=False
)

# Add the sequence values trace on the secondary y-axis.
fig.add_trace(
    go.Scatter(
        x=event_explanation["Event Number"],
        y=sequence_line,
        mode='lines',
        name='Sequence Values'
    ),
    secondary_y=True
)

# Update layout and y-axes titles.
fig.update_layout(
    title="Event-level Explanation with Dual Y-Axis",
    xaxis_title="Event Number"
)
fig.update_yaxes(title_text="Shapley Value", secondary_y=False)
fig.update_yaxes(title_text="Sequence Value", secondary_y=True)

fig.show()

In [30]:
# Make predictions on the test set
y_pred = model.predict(random_sequence)

# Rescale predictions and test data back to the original range
y_pred_rescaled = y_pred.flatten() * y_max

y_pred_rescaled

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


array([371.68304], dtype=float32)